# Exit Attribution + Crowding / Factor-Decay Anchors (DualMoM & ST-MoM)

このノートブックは、添付の `p00_dualmom.ipynb` / `p01_stmom.ipynb` と **同一の月次パネル・形成月→保有月のタイミング**に整合する形で、

- **Exit event**（どの銘柄がいつ外れたか）
- **Exit reason**（戦略ロジックに基づく単一タグ）
- **Crowding（混雑）アンカー**（相関・第一固有値比率）
- **Factor decay（効きの減衰）アンカー**（Rank IC・Dispersion）
- 図表（Holdings matrix / Exit reason / Regime map / Event study / Survival）

を生成し、`outputs/` に CSV/PNG を保存します。


In [ ]:
from dataclasses import dataclass

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# =========================
# 1) Load data (same as p00/p01)
# =========================
df = pd.read_csv("data/us_sector_etf_9_ohlc.csv", header=[0, 1], index_col=0)
df.index = pd.to_datetime(df.index)

adj_close = df["Adj Close"].ffill()
volume = df["Volume"].astype(float)

START_DATE = df.index[0]
COST_BPS = 10
TOP_K = 3

print("START_DATE:", START_DATE)
print("tickers:", list(adj_close.columns))

In [ ]:
# =========================
# 2) Monthly builders (copied from p00/p01)
# =========================
def rolling_zscore(df: pd.DataFrame, window: int = 12, min_periods: int = 12) -> pd.DataFrame:
    mu = df.rolling(window=window, min_periods=min_periods).mean()
    sd = df.rolling(window=window, min_periods=min_periods).std(ddof=1)
    return (df - mu) / sd

def monthly_from_daily_dual(adj_close_daily: pd.DataFrame, volume_daily: pd.DataFrame):
    monthly_price = adj_close_daily.resample('ME').last()
    monthly_ret = monthly_price.pct_change()
    monthly_vol_sum = volume_daily.resample('ME').sum(min_count=1)
    monthly_mom = monthly_price.shift(2) / monthly_price.shift(12) - 1
    return monthly_price, monthly_ret, monthly_vol_sum, monthly_mom

def monthly_from_daily_st(adj_close_daily: pd.DataFrame, volume_daily: pd.DataFrame):
    monthly_price = adj_close_daily.resample('ME').last()
    monthly_ret = monthly_price.pct_change()
    monthly_vol_sum = volume_daily.resample('ME').sum(min_count=1)
    turnover_base = np.log(monthly_vol_sum.replace(0, np.nan))
    turnover_proxy = rolling_zscore(turnover_base, window=12, min_periods=12)
    return monthly_price, monthly_ret, monthly_vol_sum, turnover_proxy

def monthly_excluding_last_3_days_dual(adj_close_daily: pd.DataFrame, volume_daily: pd.DataFrame):
    idx = adj_close_daily.index
    month_key = idx.to_period('M')
    rank_from_end = idx.to_series(index=idx).groupby(month_key).cumcount(ascending=False) + 1
    keep_mask = rank_from_end > 3
    adj_keep = adj_close_daily.loc[keep_mask.values].copy()
    vol_keep = volume_daily.loc[keep_mask.values].copy()
    month_key_keep = adj_keep.index.to_period('M')
    price_eom = adj_keep.groupby(month_key_keep).last()
    vol_eom = vol_keep.groupby(month_key_keep).sum(min_count=1)
    price_eom.index = price_eom.index.to_timestamp(how='end').normalize()
    vol_eom.index = vol_eom.index.to_timestamp(how='end').normalize()
    ret_eom = price_eom.pct_change()
    mom_eom = price_eom.shift(2) / price_eom.shift(12) - 1
    return price_eom, ret_eom, vol_eom, mom_eom

def monthly_excluding_last_3_days_st(adj_close_daily: pd.DataFrame, volume_daily: pd.DataFrame):
    idx = adj_close_daily.index
    month_key = idx.to_period('M')
    rank_from_end = idx.to_series(index=idx).groupby(month_key).cumcount(ascending=False) + 1
    keep_mask = rank_from_end > 3
    adj_keep = adj_close_daily.loc[keep_mask.values].copy()
    vol_keep = volume_daily.loc[keep_mask.values].copy()
    month_key_keep = adj_keep.index.to_period('M')
    price_eom = adj_keep.groupby(month_key_keep).last()
    vol_eom = vol_keep.groupby(month_key_keep).sum(min_count=1)
    price_eom.index = price_eom.index.to_timestamp(how='end').normalize()
    vol_eom.index = vol_eom.index.to_timestamp(how='end').normalize()
    ret_eom = price_eom.pct_change()
    turnover_base = np.log(vol_eom.replace(0, np.nan))
    turnover_proxy = rolling_zscore(turnover_base, window=12, min_periods=12)
    return price_eom, ret_eom, vol_eom, turnover_proxy

## 3) Strategy builders（p00/p01と同一ロジック）

In [ ]:
@dataclass
class StrategyResult:
    name: str
    ret_gross: pd.Series
    turnover: pd.Series
    ret_net: pd.Series
    weights: pd.DataFrame

def build_eqw_strategy(
    monthly_ret: pd.DataFrame,
    strategy_name: str,
    cost_bps: float = 10.0,
) -> StrategyResult:
    months = monthly_ret.index
    assets = list(monthly_ret.columns)

    w_list = []
    gross = []
    one_way_turn = []
    prev_w = pd.Series(0.0, index=assets)

    for i in range(len(months) - 1):
        m = months[i]
        h = months[i + 1]

        r_form = monthly_ret.loc[m]
        r_hold = monthly_ret.loc[h]

        valid = r_hold.notna()
        if valid.sum() < 4:
            continue

        rv = r_form[valid]
        w_assets = rv.index

        if len(w_assets) == 0:
            continue

        w = pd.Series(0.0, index=assets)
        w.loc[w_assets] = 1.0 / len(w_assets)

        assert h > m, f'holding month must be after formation month: {m} -> {h}'

        g = float((w * r_hold).sum())
        t = float(0.5 * (w - prev_w).abs().sum())
        n = g - t * (cost_bps / 10000.0)

        w_list.append(w.rename(h))
        gross.append((h, g))
        one_way_turn.append((h, t))
        prev_w = w
    
    if not w_list:
        raise ValueError(f'No valid monthly portfolios for {strategy_name}')

    W = pd.DataFrame(w_list)
    ret_gross = pd.Series(dict(gross), name=f'{strategy_name}_gross').sort_index()
    turn = pd.Series(dict(one_way_turn), name=f'{strategy_name}_turnover').sort_index()
    ret_net = (ret_gross - turn * (cost_bps / 10000.0)).rename(f'{strategy_name}_net')

    diff = (ret_gross - ret_net - turn * (cost_bps / 10000.0)).abs().max()
    assert diff < 1e-12, f'cost consistency failed for {strategy_name}: {diff}'

    return StrategyResult(strategy_name, ret_gross, turn, ret_net, W.sort_index())

def build_mom_strategy(
    monthly_ret: pd.DataFrame,
    momentum_signal: pd.DataFrame,
    strategy_name: str,
    momentum_high: bool = True,
    top_k: int = 3,
    cost_bps: float = 10.0,
) -> StrategyResult:
    months = monthly_ret.index
    assets = list(monthly_ret.columns)

    w_list = []
    gross = []
    one_way_turn = []
    prev_w = pd.Series(0.0, index=assets)

    for i in range(len(months) - 1):
        m = months[i]
        h = months[i + 1]

        r_form = monthly_ret.loc[m]
        m_form = momentum_signal.loc[m]
        r_hold = monthly_ret.loc[h]

        valid = r_form.notna() & m_form.notna() & r_hold.notna()
        if valid.sum() < 4:
            continue
        
        score = m_form[valid].copy()
        abs_score = m_form[valid].copy()
        score = score.where(abs_score > 0)
        
        w_assets = score.sort_values(ascending=False).dropna().head(top_k).index
        l_assets = score.sort_values(ascending=False).dropna().tail(top_k).index

        if len(w_assets) == 0 or len(l_assets) == 0:
            continue

        w = pd.Series(0.0, index=assets)
        if momentum_high:
            w.loc[w_assets] = 1.0 / len(w_assets)
        else:
            w.loc[l_assets] = 1.0 / len(l_assets)

        assert h > m, f'holding month must be after formation month: {m} -> {h}'

        g = float((w * r_hold).sum())
        t = float(0.5 * (w - prev_w).abs().sum())
        n = g - t * (cost_bps / 10000.0)

        w_list.append(w.rename(h))
        gross.append((h, g))
        one_way_turn.append((h, t))
        prev_w = w

    if not w_list:
        raise ValueError(f'No valid monthly portfolios for {strategy_name}')

    W = pd.DataFrame(w_list)
    ret_gross = pd.Series(dict(gross), name=f'{strategy_name}_gross').sort_index()
    turn = pd.Series(dict(one_way_turn), name=f'{strategy_name}_turnover').sort_index()
    ret_net = (ret_gross - turn * (cost_bps / 10000.0)).rename(f'{strategy_name}_net')

    diff = (ret_gross - ret_net - turn * (cost_bps / 10000.0)).abs().max()
    assert diff < 1e-12, f'cost consistency failed for {strategy_name}: {diff}'

    return StrategyResult(strategy_name, ret_gross, turn, ret_net, W.sort_index())

eqw = build_eqw_strategy(
    monthly_ret = monthly_ret,
    strategy_name = 'EQW',
    cost_bps = COST_BPS,
)

mom_w = build_mom_strategy(
    monthly_ret = monthly_ret,
    momentum_signal = momentum_proxy,
    strategy_name = 'DualMoM_W',
    momentum_high = True,
    top_k = TOP_K,
    cost_bps=COST_BPS,
)

mom_l = build_mom_strategy(
    monthly_ret = monthly_ret,
    momentum_signal = momentum_proxy,
    strategy_name = 'DualMoM_L',
    momentum_high = False,
    top_k = TOP_K,
    cost_bps=COST_BPS,
)

In [ ]:
def build_eqw_strategy(
    monthly_ret: pd.DataFrame,
    strategy_name: str,
    cost_bps: float = 10.0,
) -> StrategyResult:
    months = monthly_ret.index
    assets = list(monthly_ret.columns)

    w_list = []
    gross = []
    one_way_turn = []
    prev_w = pd.Series(0.0, index=assets)

    for i in range(len(months) - 1):
        m = months[i]
        h = months[i + 1]

        r_form = monthly_ret.loc[m]
        r_hold = monthly_ret.loc[h]

        valid = r_hold.notna()
        if valid.sum() < 4:
            continue

        rv = r_form[valid]
        w_assets = rv.index

        if len(w_assets) == 0:
            continue

        w = pd.Series(0.0, index=assets)
        w.loc[w_assets] = 1.0 / len(w_assets)

        assert h > m, f'holding month must be after formation month: {m} -> {h}'

        g = float((w * r_hold).sum())
        t = float(0.5 * (w - prev_w).abs().sum())
        n = g - t * (cost_bps / 10000.0)

        w_list.append(w.rename(h))
        gross.append((h, g))
        one_way_turn.append((h, t))
        prev_w = w
    
    if not w_list:
        raise ValueError(f'No valid monthly portfolios for {strategy_name}')

    W = pd.DataFrame(w_list)
    ret_gross = pd.Series(dict(gross), name=f'{strategy_name}_gross').sort_index()
    turn = pd.Series(dict(one_way_turn), name=f'{strategy_name}_turnover').sort_index()
    ret_net = (ret_gross - turn * (cost_bps / 10000.0)).rename(f'{strategy_name}_net')

    diff = (ret_gross - ret_net - turn * (cost_bps / 10000.0)).abs().max()
    assert diff < 1e-12, f'cost consistency failed for {strategy_name}: {diff}'

    return StrategyResult(strategy_name, ret_gross, turn, ret_net, W.sort_index())

def build_stmom_strategy(
    monthly_ret: pd.DataFrame,
    turnover_signal: pd.DataFrame,
    strategy_name: str,
    momentum_high: bool = True,
    turnover_high: bool = True,
    cost_bps: float = 10.0,
) -> StrategyResult:
    months = monthly_ret.index
    assets = list(monthly_ret.columns)

    w_list = []
    gross = []
    one_way_turn = []
    prev_w = pd.Series(0.0, index=assets)

    for i in range(len(months) - 1):
        m = months[i]
        h = months[i + 1]

        r_form = monthly_ret.loc[m]
        t_form = turnover_signal.loc[m]
        r_hold = monthly_ret.loc[h]

        valid = r_form.notna() & t_form.notna() & r_hold.notna()
        if valid.sum() < 4:
            continue

        rv = r_form[valid]
        tv = t_form[valid]

        r_med = rv.median()
        t_med = tv.median()

        winners = rv.index[rv >= r_med]
        losers = rv.index[rv < r_med]

        if turnover_high:
            high_turn = tv.index[tv >= t_med]
            w_assets = list(set(winners).intersection(high_turn))
            l_assets = list(set(losers).intersection(high_turn))
        else:
            low_turn = tv.index[tv < t_med]
            w_assets = list(set(winners).intersection(low_turn))
            l_assets = list(set(losers).intersection(low_turn))

        if len(w_assets) == 0 or len(l_assets) == 0:
            continue

        w = pd.Series(0.0, index=assets)
        if momentum_high:
            w.loc[w_assets] = 1.0 / len(w_assets)
        else:
            w.loc[l_assets] = 1.0 / len(l_assets)

        assert h > m, f'holding month must be after formation month: {m} -> {h}'

        g = float((w * r_hold).sum())
        t = float(0.5 * (w - prev_w).abs().sum())
        n = g - t * (cost_bps / 10000.0)

        w_list.append(w.rename(h))
        gross.append((h, g))
        one_way_turn.append((h, t))
        prev_w = w

    if not w_list:
        raise ValueError(f'No valid monthly portfolios for {strategy_name}')

    W = pd.DataFrame(w_list)
    ret_gross = pd.Series(dict(gross), name=f'{strategy_name}_gross').sort_index()
    turn = pd.Series(dict(one_way_turn), name=f'{strategy_name}_turnover').sort_index()
    ret_net = (ret_gross - turn * (cost_bps / 10000.0)).rename(f'{strategy_name}_net')

    diff = (ret_gross - ret_net - turn * (cost_bps / 10000.0)).abs().max()
    assert diff < 1e-12, f'cost consistency failed for {strategy_name}: {diff}'

    return StrategyResult(strategy_name, ret_gross, turn, ret_net, W.sort_index())

eqw = build_eqw_strategy(
    monthly_ret=monthly_ret,
    strategy_name='EQW',
    cost_bps=COST_BPS,
)

stmom_wh = build_stmom_strategy(
    monthly_ret=monthly_ret,
    turnover_signal=turnover_proxy,
    strategy_name='STMOM_WH',
    momentum_high=True,
    turnover_high=True,
    cost_bps=COST_BPS,
)

stmom_wl = build_stmom_strategy(
    monthly_ret=monthly_ret,
    turnover_signal=turnover_proxy,
    strategy_name='STMOM_WL',
    momentum_high=True,
    turnover_high=False,
    cost_bps=COST_BPS,
)

stmom_lh = build_stmom_strategy(
    monthly_ret=monthly_ret,
    turnover_signal=turnover_proxy,
    strategy_name='STMOM_LH',
    momentum_high=False,
    turnover_high=True,
    cost_bps=COST_BPS,
)

stmom_ll = build_stmom_strategy(
    monthly_ret=monthly_ret,
    turnover_signal=turnover_proxy,
    strategy_name='STMOM_LL',
    momentum_high=False,
    turnover_high=False,
    cost_bps=COST_BPS,
)

In [ ]:
def annualized_metrics(ret: pd.Series, periods_per_year: int = 12) -> dict:
    ret = ret.dropna()
    if len(ret) == 0:
        return {'n_months': 0, 'CAGR': np.nan, 'Vol': np.nan, 'Sharpe': np.nan, 'MaxDD': np.nan, 'HitRate': np.nan}

    equity = (1 + ret).cumprod()
    years = len(ret) / periods_per_year
    cagr = equity.iloc[-1] ** (1 / years) - 1 if years > 0 else np.nan
    vol = ret.std(ddof=1) * np.sqrt(periods_per_year)
    sharpe = (ret.mean() * periods_per_year) / vol if vol and vol > 0 else np.nan
    dd = equity / equity.cummax() - 1
    maxdd = dd.min()
    hit = (ret > 0).mean()

    return {
        'n_months': int(len(ret)),
        'CAGR': float(cagr),
        'Vol': float(vol),
        'Sharpe': float(sharpe),
        'MaxDD': float(maxdd),
        'HitRate': float(hit),
    }

strategy_returns = pd.concat([
    eqw.ret_net,
    mom_w.ret_net,
    mom_l.ret_net,
], axis=1).sort_index()

metrics_rows = []
for c in strategy_returns.columns:
    if c.endswith('_gross') or c.endswith('_net'):
        m = annualized_metrics(strategy_returns[c])
        m['series'] = c
        metrics_rows.append(m)

metrics_summary = pd.DataFrame(metrics_rows).set_index('series').sort_index()
metrics_summary

def plot_equity_curve(ret_series: pd.Series, title: str):
    ret_series = ret_series.dropna()
    eq = (1 + ret_series.dropna()).cumprod()
    fig, ax = plt.subplots(figsize=(10, 4))
    for item in eq.columns:
        ax.plot(eq[item], label=item)
    ax.set_title(title)
    ax.set_ylabel('Cumulative Growth')
    ax.grid(True, alpha=0.3)
    ax.legend()
    fig.tight_layout()
    plt.show()

def plot_weights(weights, title):
    fig, ax = plt.subplots(figsize=(10, 4), tight_layout=True)
    ax.set_title(title)
    ax.stackplot(weights.index, weights.T, labels=weights.columns)
    ax.legend()
    plt.show()

plot_equity_curve(strategy_returns, 'Net Equity Curve')
plot_weights(mom_w.weights, 'MOM ETF Gross Equity Curve')

## 4) Run strategies (DualMoM / ST-MoM)

In [ ]:
# =========================
# 4) Run strategies
# =========================
monthly_price, monthly_ret, monthly_vol, momentum_proxy = monthly_from_daily_dual(adj_close, volume)
_, _, _, _ = monthly_excluding_last_3_days_dual(adj_close, volume)  # not used in this notebook; kept for parity

# ST uses the same monthly_ret built above (price-based) for consistency with p01
# but turnover proxy is computed per p01 logic:
_, monthly_ret_st, monthly_vol_st, turnover_proxy = monthly_from_daily_st(adj_close, volume)

assert monthly_ret.index.equals(monthly_ret_st.index), "monthly_ret mismatch between dual and st builders"

res_dual = build_mom_strategy(
    monthly_ret=monthly_ret,
    momentum_signal=momentum_proxy,
    strategy_name="DualMoM",
    momentum_high=True,
    top_k=TOP_K,
    cost_bps=COST_BPS,
)
res_st = build_stmom_strategy(
    monthly_ret=monthly_ret,
    turnover_proxy=turnover_proxy,
    strategy_name="STMoM",
    momentum_high=True,
    turnover_high=True,
    cost_bps=COST_BPS,
)

W_dual = res_dual.weights.copy()
W_st = res_st.weights.copy()

print("DualMoM weights shape:", W_dual.shape, "from", W_dual.index.min(), "to", W_dual.index.max())
print("STMoM  weights shape:", W_st.shape, "from", W_st.index.min(), "to", W_st.index.max())

## 5) Exit events / Exit reasons / Anchors

In [ ]:
# =========================
# 5) Core utilities
# =========================
def ensure_outputs_dir(path: str = "outputs"):
    os.makedirs(path, exist_ok=True)
    return path

def compute_in_port(W: pd.DataFrame, eps: float = 0.0) -> pd.DataFrame:
    return (W.fillna(0.0) > eps)

def compute_exit_events(W: pd.DataFrame) -> pd.DataFrame:
    in_port = compute_in_port(W)
    exit_df = pd.DataFrame(False, index=W.index, columns=W.columns)
    prev = in_port.shift(1)
    exit_df.loc[:, :] = (prev.fillna(False) & (~in_port))
    return exit_df

def iqr(x: pd.Series) -> float:
    x = x.dropna()
    if len(x) == 0:
        return np.nan
    return float(x.quantile(0.75) - x.quantile(0.25))

def spearman_corr(a: pd.Series, b: pd.Series) -> float:
    df = pd.concat([a, b], axis=1).dropna()
    if len(df) < 3:
        return np.nan
    return float(df.iloc[:, 0].rank().corr(df.iloc[:, 1].rank()))

def avg_offdiag_corr(R: pd.DataFrame) -> float:
    # R: T x N returns
    if R.shape[1] < 2:
        return np.nan
    C = R.corr()
    vals = C.values
    n = vals.shape[0]
    off = vals[~np.eye(n, dtype=bool)]
    if off.size == 0:
        return np.nan
    return float(np.nanmean(off))

def eig1_share(R: pd.DataFrame) -> float:
    # Use covariance; returns window T x N
    if R.shape[1] < 2:
        return np.nan
    X = R.dropna(axis=0, how="any")
    if len(X) < 3:
        return np.nan
    cov = X.cov().values
    try:
        w = np.linalg.eigvalsh(cov)
    except np.linalg.LinAlgError:
        return np.nan
    s = float(np.nansum(w))
    if not np.isfinite(s) or s <= 0:
        return np.nan
    return float(np.nanmax(w) / s)

In [ ]:
# =========================
# 6A) DualMoM: formation mapping + selection + exit reasons
# =========================
def dual_formation_mapping_and_selection(monthly_ret: pd.DataFrame, momentum_signal: pd.DataFrame, top_k: int = 3):
    months = monthly_ret.index
    assets = list(monthly_ret.columns)
    # Map hold month -> formation month used
    hold_to_form = {}
    # For each hold month, store eligible/select/score_raw (formation-month based)
    score_raw_by_hold = {}
    eligible_by_hold = {}
    selected_by_hold = {}
    valid_by_hold = {}

    for i in range(len(months) - 1):
        m = months[i]
        h = months[i + 1]
        r_form = monthly_ret.loc[m]
        m_form = momentum_signal.loc[m]
        r_hold = monthly_ret.loc[h]
        valid = r_form.notna() & m_form.notna() & r_hold.notna()
        if valid.sum() < 4:
            continue

        score_raw = m_form[valid].copy()
        eligible = (score_raw > 0)
        score_pos = score_raw.where(eligible)

        if score_pos.dropna().empty:
            selected = pd.Series(False, index=assets)
        else:
            chosen = score_pos.sort_values(ascending=False).dropna().head(top_k).index
            selected = pd.Series(False, index=assets)
            selected.loc[chosen] = True

        hold_to_form[h] = m
        sr = pd.Series(np.nan, index=assets); sr.loc[score_raw.index] = score_raw.values
        el = pd.Series(False, index=assets); el.loc[score_raw.index] = eligible.values
        va = pd.Series(False, index=assets); va.loc[valid.index] = valid.values

        score_raw_by_hold[h] = sr
        eligible_by_hold[h] = el
        selected_by_hold[h] = selected
        valid_by_hold[h] = va

    score_raw_df = pd.DataFrame(score_raw_by_hold).T.reindex(columns=assets)
    eligible_df = pd.DataFrame(eligible_by_hold).T.reindex(columns=assets)
    selected_df = pd.DataFrame(selected_by_hold).T.reindex(columns=assets)
    valid_df = pd.DataFrame(valid_by_hold).T.reindex(columns=assets)
    return hold_to_form, score_raw_df, eligible_df, selected_df, valid_df

def dual_exit_reasons(W: pd.DataFrame, monthly_ret: pd.DataFrame, momentum_signal: pd.DataFrame, top_k: int = 3):
    assets = list(W.columns)
    exit_df = compute_exit_events(W)

    hold_to_form, score_raw_h, eligible_h, selected_h, valid_h = dual_formation_mapping_and_selection(
        monthly_ret=monthly_ret, momentum_signal=momentum_signal, top_k=top_k
    )

    reasons = pd.DataFrame("", index=W.index, columns=assets)
    for h in W.index:
        if h not in hold_to_form:
            # weights exist but mapping missing => mark as DATA_OUT for any exits
            for a in assets:
                if bool(exit_df.loc[h, a]):
                    reasons.loc[h, a] = "DATA_OUT"
            continue

        # formation month is hold_to_form[h], but we already stored formation diagnostics keyed by hold-month h
        for a in assets:
            if not bool(exit_df.loc[h, a]):
                continue
            if not bool(valid_h.loc[h, a]):
                reasons.loc[h, a] = "DATA_OUT"
            elif not bool(eligible_h.loc[h, a]):
                reasons.loc[h, a] = "ABS_FAIL"
            elif not bool(selected_h.loc[h, a]):
                reasons.loc[h, a] = "RANK_OUT"
            else:
                reasons.loc[h, a] = "UNEXPECTED"
    return exit_df, reasons, hold_to_form, score_raw_h, selected_h

In [ ]:
# =========================
# 6B) ST-MoM: formation mapping + buckets + exit reasons
# =========================
def st_formation_mapping_and_buckets(monthly_ret: pd.DataFrame, turnover_proxy: pd.DataFrame):
    months = monthly_ret.index
    assets = list(monthly_ret.columns)
    hold_to_form = {}
    is_winner_by_hold = {}
    is_highturn_by_hold = {}
    valid_by_hold = {}
    r_form_by_hold = {}
    t_form_by_hold = {}

    for i in range(len(months) - 1):
        m = months[i]
        h = months[i + 1]
        r_form = monthly_ret.loc[m]
        t_form = turnover_proxy.loc[m]
        r_hold = monthly_ret.loc[h]

        valid = r_form.notna() & t_form.notna() & r_hold.notna()
        if valid.sum() < 4:
            continue

        rv = r_form[valid].copy()
        tv = t_form[valid].copy()
        r_med = float(rv.median())
        t_med = float(tv.median())

        is_w = (rv >= r_med)
        is_ht = (tv >= t_med)

        iw = pd.Series(False, index=assets); iw.loc[rv.index] = is_w.values
        iht = pd.Series(False, index=assets); iht.loc[tv.index] = is_ht.values
        va = pd.Series(False, index=assets); va.loc[valid.index] = valid.values
        rr = pd.Series(np.nan, index=assets); rr.loc[rv.index] = rv.values
        tt = pd.Series(np.nan, index=assets); tt.loc[tv.index] = tv.values

        hold_to_form[h] = m
        is_winner_by_hold[h] = iw
        is_highturn_by_hold[h] = iht
        valid_by_hold[h] = va
        r_form_by_hold[h] = rr
        t_form_by_hold[h] = tt

    is_winner_h = pd.DataFrame(is_winner_by_hold).T.reindex(columns=assets)
    is_highturn_h = pd.DataFrame(is_highturn_by_hold).T.reindex(columns=assets)
    valid_h = pd.DataFrame(valid_by_hold).T.reindex(columns=assets)
    r_form_h = pd.DataFrame(r_form_by_hold).T.reindex(columns=assets)
    t_form_h = pd.DataFrame(t_form_by_hold).T.reindex(columns=assets)
    return hold_to_form, is_winner_h, is_highturn_h, valid_h, r_form_h, t_form_h

def st_exit_reasons(W: pd.DataFrame, monthly_ret: pd.DataFrame, turnover_proxy: pd.DataFrame):
    assets = list(W.columns)
    exit_df = compute_exit_events(W)

    hold_to_form, is_w_h, is_ht_h, valid_h, r_form_h, t_form_h = st_formation_mapping_and_buckets(
        monthly_ret=monthly_ret, turnover_proxy=turnover_proxy
    )

    reasons = pd.DataFrame("", index=W.index, columns=assets)

    # To define flip we compare bucket states between consecutive hold months in W.index
    prev_h = None
    for h in W.index:
        for a in assets:
            if not bool(exit_df.loc[h, a]):
                continue

            if (h not in hold_to_form) or (not bool(valid_h.loc[h, a])):
                reasons.loc[h, a] = "DATA_OUT"
                continue

            if prev_h is None or (prev_h not in hold_to_form) or (not bool(valid_h.loc[prev_h, a])):
                # no prior state to compare; fall back to bucket membership at h
                # If it exited, it must no longer satisfy the selected cell (winners/high_turn config) or data issue
                reasons.loc[h, a] = "BUCKET_CHANGE"
                continue

            flip_mom = bool(is_w_h.loc[h, a]) != bool(is_w_h.loc[prev_h, a])
            flip_turn = bool(is_ht_h.loc[h, a]) != bool(is_ht_h.loc[prev_h, a])

            if flip_mom and flip_turn:
                reasons.loc[h, a] = "BOTH_FLIP"
            elif flip_mom:
                reasons.loc[h, a] = "MOM_BUCKET_FLIP"
            elif flip_turn:
                reasons.loc[h, a] = "TURN_BUCKET_FLIP"
            else:
                reasons.loc[h, a] = "UNEXPECTED"
        prev_h = h

    return exit_df, reasons, hold_to_form, r_form_h, t_form_h

In [ ]:
# =========================
# 7) Anchors (crowding / factor-decay) aligned to formation month m via hold-month h
# =========================
def compute_anchors_dual(W: pd.DataFrame, monthly_ret: pd.DataFrame, momentum_signal: pd.DataFrame, top_k: int = 3, corr_window: int = 12):
    exit_df, reasons, hold_to_form, score_raw_h, selected_h = dual_exit_reasons(W, monthly_ret, momentum_signal, top_k=top_k)

    # Formation-month keyed (we store by hold-month h; formation is hold_to_form[h])
    rows = []
    for h in W.index:
        if h not in hold_to_form:
            continue
        m = hold_to_form[h]
        sel_assets = list(selected_h.columns[selected_h.loc[h].fillna(False).values])
        # Crowding window ends at m (formation month)
        if m not in monthly_ret.index:
            continue
        mi = monthly_ret.index.get_loc(m)
        lo = max(0, mi - corr_window + 1)
        win = monthly_ret.iloc[lo:mi+1, :]

        R = win[sel_assets].dropna(axis=1, how="all")
        crowd_corr = avg_offdiag_corr(R) if len(sel_assets) >= 2 else np.nan
        crowd_eig = eig1_share(R) if len(sel_assets) >= 2 else np.nan

        # Factor efficacy: score_raw at formation vs next-month return (hold month h)
        sr = score_raw_h.loc[h]
        rh = monthly_ret.loc[h] if h in monthly_ret.index else np.nan
        ic = spearman_corr(sr, rh)
        disp = iqr(sr)

        rows.append({"hold_month": h, "form_month": m, "n_sel": len(sel_assets),
                     "crowd_avg_corr": crowd_corr, "crowd_eig1_share": crowd_eig,
                     "rank_ic": ic, "dispersion_iqr": disp})

    anchors = pd.DataFrame(rows).set_index("hold_month").sort_index()
    return exit_df, reasons, anchors

def compute_anchors_st(W: pd.DataFrame, monthly_ret: pd.DataFrame, turnover_proxy: pd.DataFrame, corr_window: int = 12):
    exit_df, reasons, hold_to_form, r_form_h, t_form_h = st_exit_reasons(W, monthly_ret, turnover_proxy)

    rows = []
    for h in W.index:
        if h not in hold_to_form:
            continue
        m = hold_to_form[h]
        sel_assets = list(W.columns[(W.loc[h].fillna(0.0) > 0).values])
        if m not in monthly_ret.index:
            continue
        mi = monthly_ret.index.get_loc(m)
        lo = max(0, mi - corr_window + 1)
        win = monthly_ret.iloc[lo:mi+1, :]

        R = win[sel_assets].dropna(axis=1, how="all")
        crowd_corr = avg_offdiag_corr(R) if len(sel_assets) >= 2 else np.nan
        crowd_eig = eig1_share(R) if len(sel_assets) >= 2 else np.nan

        r_score = r_form_h.loc[h]
        t_score = t_form_h.loc[h]
        rh = monthly_ret.loc[h] if h in monthly_ret.index else np.nan

        ic_ret = spearman_corr(r_score, rh)
        ic_turn = spearman_corr(t_score, rh)
        disp_ret = iqr(r_score)
        disp_turn = iqr(t_score)

        rows.append({"hold_month": h, "form_month": m, "n_sel": len(sel_assets),
                     "crowd_avg_corr": crowd_corr, "crowd_eig1_share": crowd_eig,
                     "rank_ic_ret": ic_ret, "rank_ic_turn": ic_turn,
                     "disp_ret_iqr": disp_ret, "disp_turn_iqr": disp_turn})

    anchors = pd.DataFrame(rows).set_index("hold_month").sort_index()
    return exit_df, reasons, anchors

In [ ]:
# =========================
# 8) Compute + export CSV
# =========================
outdir = ensure_outputs_dir("outputs")

exit_dual, reason_dual, anchors_dual = compute_anchors_dual(W_dual, monthly_ret, momentum_proxy, top_k=TOP_K, corr_window=12)
exit_st, reason_st, anchors_st = compute_anchors_st(W_st, monthly_ret, turnover_proxy, corr_window=12)

def make_summary(W: pd.DataFrame, exit_df: pd.DataFrame, res: StrategyResult):
    in_port = compute_in_port(W)
    holdings_prev = in_port.shift(1).sum(axis=1)
    exit_count = exit_df.sum(axis=1)
    exit_rate = (exit_count / holdings_prev).replace([np.inf, -np.inf], np.nan)
    summary = pd.DataFrame({
        "holdings_prev": holdings_prev,
        "exit_count": exit_count,
        "exit_rate": exit_rate,
        "turnover": res.turnover.reindex(W.index),
        "ret_gross": res.ret_gross.reindex(W.index),
        "ret_net": res.ret_net.reindex(W.index),
    })
    return summary

summary_dual = make_summary(W_dual, exit_dual, res_dual)
summary_st = make_summary(W_st, exit_st, res_st)

# Save (facts)
exit_dual.astype(int).to_csv(os.path.join(outdir, "exit_events_dual.csv"))
reason_dual.to_csv(os.path.join(outdir, "exit_reasons_dual.csv"))
anchors_dual.to_csv(os.path.join(outdir, "anchors_dual.csv"))
summary_dual.to_csv(os.path.join(outdir, "summary_dual.csv"))

exit_st.astype(int).to_csv(os.path.join(outdir, "exit_events_st.csv"))
reason_st.to_csv(os.path.join(outdir, "exit_reasons_st.csv"))
anchors_st.to_csv(os.path.join(outdir, "anchors_st.csv"))
summary_st.to_csv(os.path.join(outdir, "summary_st.csv"))

print("Saved CSVs under:", outdir)
display(summary_dual.tail(5))
display(summary_st.tail(5))

In [ ]:
# =========================
# 9) Plot helpers
# =========================
def plot_holdings_matrix(W: pd.DataFrame, exit_df: pd.DataFrame, reasons: pd.DataFrame, title: str, path_png: str):
    W0 = W.fillna(0.0)
    fig, ax = plt.subplots(figsize=(12, 6))
    im = ax.imshow(W0.values, aspect="auto", interpolation="nearest")
    ax.set_title(title)
    ax.set_yticks(range(len(W0.index)))
    ax.set_yticklabels([d.strftime("%Y-%m") for d in W0.index], fontsize=8)
    ax.set_xticks(range(len(W0.columns)))
    ax.set_xticklabels(W0.columns, rotation=90, fontsize=8)

    # Overlay exit markers (marker shapes encode reason)
    marker_map = {
        "ABS_FAIL": "x",
        "RANK_OUT": "o",
        "DATA_OUT": "s",
        "MOM_BUCKET_FLIP": "x",
        "TURN_BUCKET_FLIP": "o",
        "BOTH_FLIP": "s",
        "BUCKET_CHANGE": "d",
        "UNEXPECTED": "*",
        "": None
    }

    ys, xs, ms = [], [], []
    for yi, h in enumerate(W0.index):
        for xi, a in enumerate(W0.columns):
            if bool(exit_df.loc[h, a]):
                r = reasons.loc[h, a]
                m = marker_map.get(r, ".")
                ys.append(yi); xs.append(xi); ms.append(m)

    # scatter per marker for legibility
    for m in sorted(set(ms)):
        pts = [(x,y) for x,y,mm in zip(xs,ys,ms) if mm==m]
        if not pts:
            continue
        px = [p[0] for p in pts]; py=[p[1] for p in pts]
        ax.scatter(px, py, marker=m, s=30, linewidths=1)

    fig.colorbar(im, ax=ax, fraction=0.02, pad=0.02)
    fig.tight_layout()
    fig.savefig(path_png, dpi=150)
    plt.close(fig)

def plot_exit_reason_timeseries(reason_df: pd.DataFrame, summary_df: pd.DataFrame, title: str, path_png: str):
    # Count exits by reason each month
    reasons = sorted([r for r in pd.unique(reason_df.values.ravel()) if isinstance(r, str) and r != ""])
    counts = {r: (reason_df == r).sum(axis=1) for r in reasons}
    idx = summary_df.index
    fig, ax = plt.subplots(figsize=(12, 4))
    bottom = np.zeros(len(idx))
    x = np.arange(len(idx))
    for r in reasons:
        y = counts[r].reindex(idx).fillna(0).values
        ax.bar(x, y, bottom=bottom, label=r)
        bottom += y

    ax2 = ax.twinx()
    ax2.plot(x, summary_df["turnover"].reindex(idx).values)

    ax.set_title(title)
    ax.set_xticks(x)
    ax.set_xticklabels([d.strftime("%Y-%m") for d in idx], rotation=90, fontsize=8)
    ax.set_ylabel("Exit count")
    ax2.set_ylabel("Turnover (one-way)")
    ax.legend(loc="upper left", fontsize=8, ncol=2)
    fig.tight_layout()
    fig.savefig(path_png, dpi=150)
    plt.close(fig)

def plot_regime_map(anchors: pd.DataFrame, summary: pd.DataFrame, x_col: str, y_col: str, c_col: str, title: str, path_png: str):
    df = anchors.join(summary[["exit_rate"]], how="left")
    df = df.dropna(subset=[x_col, y_col])
    x = df[x_col].values
    y = df[y_col].values
    s = (df["exit_rate"].fillna(0).values * 800) + 20
    c = df[c_col].values if c_col in df.columns else None

    fig, ax = plt.subplots(figsize=(6, 5))
    sc = ax.scatter(x, y, s=s, c=c)
    ax.set_title(title)
    ax.set_xlabel(x_col)
    ax.set_ylabel(y_col)
    if c is not None:
        fig.colorbar(sc, ax=ax, fraction=0.04, pad=0.02, label=c_col)
    fig.tight_layout()
    fig.savefig(path_png, dpi=150)
    plt.close(fig)

def plot_event_study(summary: pd.DataFrame, anchors: pd.DataFrame, event_months: pd.DatetimeIndex, cols: list, window: int, title: str, path_png: str):
    # Align series on event index 0 and average
    idx = summary.index
    pos = {d:i for i,d in enumerate(idx)}
    rows = []
    for h in event_months:
        if h not in pos:
            continue
        i0 = pos[h]
        lo = max(0, i0-window)
        hi = min(len(idx)-1, i0+window)
        rel = np.arange(lo, hi+1) - i0
        block = pd.DataFrame(index=rel)
        for col in cols:
            if col in summary.columns:
                s = summary[col]
            else:
                s = anchors[col]
            block[col] = s.reindex(idx[lo:hi+1]).values
        rows.append(block)
    if not rows:
        return
    # average on relative index
    rel_index = sorted(set(np.concatenate([r.index.values for r in rows])))
    avg = pd.DataFrame(index=rel_index)
    for col in cols:
        mats = []
        for r in rows:
            mats.append(r[col].reindex(rel_index).values)
        avg[col] = np.nanmean(np.vstack(mats), axis=0)

    fig, ax = plt.subplots(figsize=(8, 4))
    for col in cols:
        ax.plot(avg.index.values, avg[col].values, label=col)
    ax.axvline(0)
    ax.set_title(title)
    ax.set_xlabel("Months relative to exit")
    ax.legend(fontsize=8, ncol=2)
    fig.tight_layout()
    fig.savefig(path_png, dpi=150)
    plt.close(fig)

def plot_survival(W: pd.DataFrame, title: str, path_png: str):
    in_port = compute_in_port(W)
    durations = []
    for a in in_port.columns:
        s = in_port[a].fillna(False).values
        run = 0
        for v in s:
            if v:
                run += 1
            else:
                if run > 0:
                    durations.append(run)
                run = 0
        if run > 0:
            durations.append(run)
    fig, ax = plt.subplots(figsize=(6,4))
    ax.hist(durations, bins=20)
    ax.set_title(title)
    ax.set_xlabel("Consecutive months in portfolio")
    ax.set_ylabel("Count")
    fig.tight_layout()
    fig.savefig(path_png, dpi=150)
    plt.close(fig)

In [ ]:
# =========================
# 10) Generate figures (PNG)
# =========================
plot_holdings_matrix(W_dual, exit_dual, reason_dual, "DualMoM Holdings Matrix (weights) + Exit markers", os.path.join(outdir, "fig01_holdings_matrix_dual.png"))
plot_holdings_matrix(W_st, exit_st, reason_st, "ST-MoM Holdings Matrix (weights) + Exit markers", os.path.join(outdir, "fig01_holdings_matrix_st.png"))

plot_exit_reason_timeseries(reason_dual, summary_dual, "DualMoM: Exit reasons (stacked) + Turnover", os.path.join(outdir, "fig02_exit_reason_timeseries_dual.png"))
plot_exit_reason_timeseries(reason_st, summary_st, "ST-MoM: Exit reasons (stacked) + Turnover", os.path.join(outdir, "fig02_exit_reason_timeseries_st.png"))

plot_regime_map(anchors_dual, summary_dual, x_col="dispersion_iqr", y_col="crowd_eig1_share", c_col="rank_ic",
                title="DualMoM Regime Map: dispersion vs crowding (size=exit_rate, color=rank_ic)",
                path_png=os.path.join(outdir, "fig03_regime_map_dual.png"))
plot_regime_map(anchors_st, summary_st, x_col="disp_ret_iqr", y_col="crowd_eig1_share", c_col="rank_ic_ret",
                title="ST-MoM Regime Map: disp_ret vs crowding (size=exit_rate, color=rank_ic_ret)",
                path_png=os.path.join(outdir, "fig03_regime_map_st.png"))

# Event study around exits (use all exit months)
event_months_dual = summary_dual.index[summary_dual["exit_count"].fillna(0) > 0]
event_months_st = summary_st.index[summary_st["exit_count"].fillna(0) > 0]

plot_event_study(summary_dual, anchors_dual, event_months_dual, cols=["turnover", "ret_net", "crowd_eig1_share", "rank_ic"], window=12,
                 title="DualMoM Exit Event Study (avg path)", path_png=os.path.join(outdir, "fig04_exit_event_study_dual.png"))
plot_event_study(summary_st, anchors_st, event_months_st, cols=["turnover", "ret_net", "crowd_eig1_share", "rank_ic_ret"], window=12,
                 title="ST-MoM Exit Event Study (avg path)", path_png=os.path.join(outdir, "fig04_exit_event_study_st.png"))

plot_survival(W_dual, "DualMoM Survival: consecutive months in portfolio", os.path.join(outdir, "fig05_survival_dual.png"))
plot_survival(W_st, "ST-MoM Survival: consecutive months in portfolio", os.path.join(outdir, "fig05_survival_st.png"))

print("Saved PNGs under:", outdir)

## 11) 主要出力の場所

- `outputs/exit_events_*.csv`：exit event（0/1）
- `outputs/exit_reasons_*.csv`：exit reason（文字列）
- `outputs/anchors_*.csv`：crowding / factor-decay アンカー
- `outputs/summary_*.csv`：exit率・ターンオーバー・リターン
- `outputs/fig01..fig05_*.png`：図表
